In [1]:
import numpy as np
import sys
import os
import random
import meshplot as mp
import igl
import import_ipynb
import scipy as sp
import trimesh
import rtree


import embedding
import extraction
import partitioning

importing Jupyter notebook from embedding.ipynb
importing Jupyter notebook from extraction.ipynb
importing Jupyter notebook from partitioning.ipynb


In [2]:
def embed(filename_in, filename_out, data, secret, strength, partitions):
    encrypted_data = encrypt(data, secret)
    
    mesh = igl.read_triangle_mesh(filename_in)
    vertices = mesh[0]        
    faces = mesh[1]

    plist = partitioning.partition(mesh, partitions)
    submeshes, mapping = partitioning.mesh_partitioning(mesh, partitions,plist)

    bits_inserted = 0
    updated_vertices = []

    print('Inserting data in ',partitions,' submeshes')
    for i, submesh in enumerate(submeshes):
        submeshVertices = submesh[0]
        submeshFaces = submesh[1]

        B = compute_eigenvectors(len(submeshVertices), submeshFaces)
        
        SV = []
        TV = []
        UV = []
        
        for i in submeshVertices:
            SV.append(i[0])
            TV.append(i[1])
            UV.append(i[2])
        
        S = np.matmul(B, SV)
        T = np.matmul(B, TV)
        U = np.matmul(B, UV)
        
        S, T, U, count = embedding.embed(S, T, U, encrypted_data, strength)
        
        BM1 = sp.linalg.pinv(B)

        new_vertices = np.zeros((len(submeshVertices), 3))
        
        new_vertices[:, 0] = np.matmul(BM1, S)
        new_vertices[:, 1] = np.matmul(BM1, T)
        new_vertices[:, 2] = np.matmul(BM1, U)
        
        updated_vertices.append([new_vertices, submeshFaces])

        bits_inserted += count

    new_vertices = np.zeros((len(vertices), 3))
    

    for i in range(len(vertices)):
        for j in range(partitions):
            val = mapping[j][i]
            if val != -1:
                new_vertices[i] = updated_vertices[j][0][int(val)]
                continue

    igl.write_triangle_mesh(filename_out,new_vertices,faces)

 

    return bits_inserted

In [3]:
def extract(filename, secret, length, strength, partitions,original_filename,allign = False):
    
    mesh = igl.read_triangle_mesh(filename)
    original_mesh = igl.read_triangle_mesh(original_filename)
    vertices = mesh[0]
    faces = mesh[1]
    if allign == True:
        tri_mesh = trimesh.Trimesh(mesh[0],mesh[1])
        other_mesh = trimesh.Trimesh(original_mesh[0],original_mesh[1])

        new_v,cost = trimesh.registration.mesh_other(other_mesh,tri_mesh, samples=500, scale=True, icp_first=10, icp_final=50)

        ok = [[new_v[0][0],new_v[0][1],new_v[0][2]],[new_v[1][0],new_v[1][1],new_v[1][2]],[new_v[2][0],new_v[2][1],new_v[2][2]]]
        vertices = np.dot(mesh[0],ok)
    mp.plot(vertices,faces)
    
    plist = partitioning.partition((vertices,faces), partitions)
    submeshes, _ = partitioning.mesh_partitioning((vertices,faces), partitions,plist)

    data = []

    for i, submesh in enumerate(submeshes):
    
        submeshVertices = submesh[0]
        submeshFaces = submesh[1]
        if len(submeshFaces) != 0:
            B = compute_eigenvectors(len(submeshVertices), submeshFaces)
            
            S = np.matmul(B, submeshVertices[:, 0])
            T = np.matmul(B, submeshVertices[:, 1])
            U = np.matmul(B, submeshVertices[:, 2])

            data.append(extraction.extract(S, T, U, strength))

    final_data = decrypt(majority_vote(data, length), secret)

    return final_data

In [4]:
def compute_eigenvectors (num_vertices, submeshes):
    submesh = submeshes.astype('int32')
    laplacian = np.zeros((num_vertices, num_vertices))
    neighbour_list = igl.adjacency_list(submesh)
    degree = []
    for i in neighbour_list:
        degree.append(len(i))
        
    for i in range(num_vertices):
        laplacian[i][i] = 1
        for j in neighbour_list[i]:
            laplacian[i][int(j)] = -1.0/degree[i]     
            
    eigenValues, eigenVectors = np.linalg.eig(laplacian)

    idx = eigenValues.argsort()
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]


    return np.transpose(eigenVectors)

In [5]:
def majority_vote(data, length):
    final_data = np.zeros(length)
    for i in range(length):
        sum = 0.0
        count = 0

        for d in data:
            j = i

            while j < len(d):
                sum += d[j]
                count += 1
                j += length

        if count > 0:
            final_data[i] = round(sum/count)

    return final_data

In [6]:
def encrypt (data, secret):
    data_copied = data[:] 
    random.Random(secret).shuffle(data_copied)
    return data_copied

In [7]:
def decrypt(encrypted_data, secret, length = 64):
    order = [*range(0,length,1)]
    random.Random(secret).shuffle(order)
    data = np.zeros(length)

    for i in range(length):
        data[order[i]] = encrypted_data[i]
    return data